# Exploring the `Last.fm` Dataset


In [ ]:
%matplotlib inline


from collections import defaultdict
from pathlib import Path
from typing import Tuple


import matplotlib as mpl
import networkx as nx
import numpy as np
import torch
from matplotlib import pyplot as plt
from scipy import sparse as sp
from sklearn.metrics import matthews_corrcoef
from sklearn.utils import check_consistent_length

SEED = 1
DATA_DIR = (
    Path().cwd().parent.joinpath("data", "processed", "lastfm", f"seed_{SEED}")
)
plt.style.use('seaborn-poster')
mpl.rcParams['figure.autolayout'] = True
assert DATA_DIR.is_dir()


## Statistics

Edges between users are undirected, and `edge_uu.txt` only stores the indices of the upper triangular part of the adjacency matrix.


In [ ]:
# `ratings`: a 2d `numpy.ndarray` object.
# Each row is a `[uid, iid, label]` triplet.
ratings = np.unique(
    np.loadtxt(DATA_DIR.joinpath("ratings.txt"), dtype=np.int64), axis=0
)
# `triplets_kg`: a 2d `numpy.ndarray` object.
# Each row is a `[eid_h, rid, eid_t]` triplet.
triplets_kg = np.unique(
    np.loadtxt(DATA_DIR.joinpath("triplets_kg.txt"), dtype=np.int64), axis=0
)
# `edges_user`: a 2d `numpy.ndarray` object.
# Each row is an unordered `[uid_u, uid_v]` pair.
edges_user = np.unique(
    np.loadtxt(DATA_DIR.joinpath("edges_uu.txt"), dtype=np.int64), axis=0
)
assert ratings.ndim == 2 and ratings.shape[1] == 3
assert triplets_kg.ndim == 2 and triplets_kg.shape[1] == 3
assert edges_user.ndim == 2 and edges_user.shape[1] == 2
# indices of the upper triangular part of the adjacency matrix
assert np.all(edges_user[:, 0] < edges_user[:, 1])
print(
    "\n".join(
        [
            f"num_ratings = {ratings.shape[0]}",
            f"num_triplets = {triplets_kg.shape[0]}",
            f"num_edges_user = {edges_user.shape[0]}",
        ]
    )
)


In [ ]:
num_users = ratings[:, 0].max() + 1
num_items = ratings[:, 1].max() + 1
num_entities = triplets_kg[:, [0, 2]].max() + 1
num_relations = triplets_kg[:, 1].max() + 1
assert num_items < num_entities
assert edges_user.max() < num_users
sparsity_ui = ratings.shape[0] / num_users / num_items
sparsity_uu = edges_user.shape[0] * 2 / num_users / (num_users - 1)
print(
    "\n".join(
        [
            f"num_users = {num_users}",
            f"num_items = {num_items}",
            f"num_entities = {num_entities}",
            f"num_relations = {num_relations}",
            f"sparsity_ui = {sparsity_ui}",
            f"sparsity_uu = {sparsity_uu}",
        ]
    )
)


## User-Item Interaction Matrix


In [ ]:
# encodes user history to a vector
# `user_history` is a `nnumpy.ndarray` object of shape `[num_users, num_items]`
# For each positive sample `(uid, iid)`, `user_history[uid, iid] = 1`.
ratings_pos = ratings[ratings[:, 2] == 1]
user_history = sp.csr_matrix(
    ([1.0] * ratings_pos.shape[0], (ratings_pos[:, 0], ratings_pos[:, 1])),
    shape=(num_users, num_items),
    dtype=np.float32,
)
user_history.nnz


In [ ]:
deg_u = user_history.sum(axis=1).A.flatten()
deg_i = user_history.sum(axis=0).A.flatten()
print(
    "\n".join(
        [
            f"deg_u: mean = {np.mean(deg_u)}, std = {np.std(deg_u)}",
            f"deg_i: mean = {np.mean(deg_i)}, std = {np.std(deg_i)}",
        ]
    )
)


In [ ]:
np.unique(deg_u, return_counts=True), np.unique(deg_i, return_counts=True)


## Knowledge Graph


In [ ]:
cnt = 0
adj_list_kg = defaultdict(list)
for eid_h, rid, eid_t in triplets_kg:
    assert eid_h < num_items
    if eid_t < num_items:
        cnt += 1
    adj_list_kg[eid_h].append((rid, eid_t))
deg_i_kg = np.asarray([len(adj_list_kg[iid]) for iid in range(num_items)])
cnt, np.unique(deg_i_kg, return_counts=True)


## Similarity between Users Connected by Social Edges


### Number of Common Neighbors & Jaccard Measure


In [ ]:
def common_neighbors_jaccard(
    y_true: sp.spmatrix, y_pred: sp.spmatrix
) -> Tuple[np.ndarray, np.ndarray]:
    assert y_true.ndim == 2 and y_pred.ndim == 2
    check_consistent_length(y_true, y_pred)
    y_true = y_true.astype(np.bool_).astype(np.int8)
    y_pred = y_pred.astype(np.bool_).astype(np.int8)
    union = y_true.multiply(y_pred)
    intersection = (y_true + y_pred).astype(np.bool_).astype(np.int8)
    num_union = union.sum(axis=1).A.astype(np.float32)
    num_intersection = intersection.sum(axis=1).A.astype(np.float32)
    return num_union, num_union / num_intersection


In [ ]:
# `common_nbrs_pos`: the number of common neighbors between users
# connected by edges
# `jaccard_pos`: the jaccard measure between users connected by edges
common_nbrs_pos, jaccard_pos = common_neighbors_jaccard(
    user_history[edges_user[:, 0], :], user_history[edges_user[:, 1], :]
)
print(
    "\n".join(
        [
            f"common_nbrs_pos: mean = {np.mean(common_nbrs_pos)}, "
            f"std = {np.std(common_nbrs_pos)}, "
            f"median = {np.median(common_nbrs_pos)}",
            f"jaccard_pos: mean = {np.mean(jaccard_pos)}, "
            f"std = {np.std(jaccard_pos)}, "
            f"median = {np.median(jaccard_pos)}",
        ]
    )
)


In [ ]:
# In the Last.fm dataset, edges are undirected.
# The number of possible edges is N = `(num_users - 1) * num_users / 2``
def encode_indices_batch(rows: np.ndarray, cols: np.ndarray) -> np.ndarray:
    # converts a `(row, col)` pair to [0, N - 1]
    assert np.all(rows < cols)
    return rows + cols * (cols - 1) // 2


def decode_indices_batch(
    indices: np.ndarray, size: int
) -> Tuple[np.ndarray, np.ndarray]:
    # converts an integer in the range [0, N - 1] to a `(row, col)` pair
    bins = np.cumsum(np.arange(size))
    cols = np.digitize(indices, bins, right=False)
    rows = indices - cols * (cols - 1) // 2
    return rows, cols


In [ ]:
indices_pos = encode_indices_batch(edges_user[:, 0], edges_user[:, 1])
assert np.unique(indices_pos).size == indices_pos.size
indices_neg = np.arange((num_users) * (num_users - 1) // 2, dtype=np.int64)
indices_neg = indices_neg[np.isin(indices_neg, indices_pos, invert=True)]
assert np.unique(indices_neg).size == indices_neg.size
rows, cols = decode_indices_batch(indices_neg, size=num_users)
assert np.all(rows < cols)
f"num_edges_user_neg = {rows.size}"


In [ ]:
# `common_nbrs_neg`: the number of common neighbors between users
# that are not connected
# `jaccard_neg`: the jaccard measure between users that are not connected
common_nbrs_neg, jaccard_neg = common_neighbors_jaccard(
    user_history[rows, :], user_history[cols, :]
)
print(
    "\n".join(
        [
            f"common_nbrs_neg: mean = {np.mean(common_nbrs_neg)}, "
            f"std = {np.std(common_nbrs_neg)}, "
            f"median = {np.median(common_nbrs_neg)}",
            f"jaccard_neg: mean = {np.mean(jaccard_neg)}, "
            f"std = {np.std(jaccard_neg)}, "
            f"median = {np.median(jaccard_neg)}",
        ]
    )
)


In [ ]:
v_max, v_min = int(common_nbrs_pos.max()), int(common_nbrs_pos.min())

figure = plt.figure()
ax = figure.add_subplot(111)
hist, bins, _ = ax.hist(
    common_nbrs_pos, bins=np.arange(v_min, v_max + 1), density=True
)
hist.sum(), hist, bins


In [ ]:
v_max, v_min = int(common_nbrs_neg.max()), int(common_nbrs_pos.min())

figure = plt.figure()
ax = figure.add_subplot(111)
hist, bins, _ = ax.hist(
    common_nbrs_neg, bins=np.arange(v_min, v_max + 1), density=True
)
hist.sum(), hist, bins


### Matthews Correlation Coefficient for Each Item


In [ ]:
mcc_per_item = np.zeros((num_items,), dtype=np.float32)
for iid in range(num_items):
    y_u = (
        user_history[edges_user[:, 0], iid].astype(np.bool_).toarray().flatten()
    )
    y_v = (
        user_history[edges_user[:, 1], iid].astype(np.bool_).toarray().flatten()
    )
    mcc_per_item[iid] = matthews_corrcoef(y_u, y_v)
mcc_per_item.max(), mcc_per_item.min()


In [ ]:
mask = deg_i >= 5
mcc_per_item_valid = mcc_per_item[mask]
mcc_per_item_valid.size


In [ ]:
figure = plt.figure()
ax = figure.add_subplot(111)
hist, bins, _ = ax.hist(
    mcc_per_item, bins=np.linspace(-0.1, 1, num=12), density=False
)
hist.sum(), hist, bins


In [ ]:
figure = plt.figure()
ax = figure.add_subplot(111)
hist, bins, _ = ax.hist(
    mcc_per_item_valid, bins=np.linspace(-0.1, 1, num=12), density=False
)
hist.sum(), hist, bins


In [ ]:
# This block is computationally expensive.
# indices_pos = encode_indices_batch(edges_user[:, 0], edges_user[:, 1])
# assert np.unique(indices_pos).size == indices_pos.size
# indices_neg = np.arange((num_users) * (num_users - 1) // 2, dtype=np.int64)
# indices_neg = indices_neg[np.isin(indices_neg, indices_pos, invert=True)]
# assert np.unique(indices_neg).size == indices_neg.size
# rows, cols = decode_indices_batch(indices_neg, size=num_users)
# assert np.all(rows < cols)


# mcc_per_item = np.zeros((num_items,), dtype=np.float32)
# for iid in range(num_items):
#     if (iid + 1) % 300 == 0:
#         print(iid)
#     y_u = (
#         user_history[rows, iid].astype(np.bool_).toarray().flatten()
#     )
#     y_v = (
#         user_history[cols, iid].astype(np.bool_).toarray().flatten()
#     )
#     mcc_per_item[iid] = matthews_corrcoef(y_u, y_v)
# mcc_per_item.max(), mcc_per_item.min()


In [ ]:
# figure = plt.figure()
# ax = figure.add_subplot(111)
# hist, bins, _ = ax.hist(
#     mcc_per_item, bins=np.linspace(-0.1, 1, num=12), density=False
# )
# hist.sum(), hist, bins


# Clusters of Each Item


In [ ]:
graph_user = nx.Graph()
graph_user.add_edges_from(edges_user)
(
    f"#nodes = {graph_user.number_of_nodes()}, "
    f"#edges = {graph_user.number_of_edges()}"
)


In [ ]:
uids_valid = set()
for comp in nx.connected_components(graph_user):
    if len(comp) > len(uids_valid):
        uids_valid = comp
uids_valid = np.asarray(sorted(uids_valid))
f"#nodes = {uids_valid.size}"


In [ ]:
# `(src, tgt)` -> length of the shortest path between `src` and `dst`
sp_len_cache = {}


def average_shortest_path_length(graph: nx.Graph, nodes: np.ndarray) -> float:
    dists = []
    for i in range(nodes.size):
        src = nodes[i]
        for j in range(i + 1, nodes.size):
            tgt = nodes[j]
            if (src, tgt) not in sp_len_cache:
                sp_len_cache[(src, tgt)] = nx.shortest_path_length(
                    graph, src, tgt
                )
            dists.append(sp_len_cache[(src, tgt)])
    return np.mean(dists)


In [ ]:
avg_dist_per_item = np.zeros((num_items,), dtype=np.float32)
deg_i_valid = np.zeros((num_items,), dtype=np.float32)


for iid in range(num_items):
    dists = []
    uids = user_history[:, iid].nonzero()[0]
    uids = uids[np.isin(uids, uids_valid)]
    deg_i_valid[iid] = uids.size
    if uids.size < 2:
        avg_dist_per_item[iid] = -1
        continue
    avg_dist_per_item[iid] = average_shortest_path_length(graph_user, uids)
avg_dist_per_item.max(), avg_dist_per_item.min()


In [ ]:
# `deg` -> average length of shortest paths between randomly selected nodes
avg_dist_per_deg = {}
avg_dist_std_per_deg = {}
num_runs = 30
for deg in np.unique(deg_i_valid):
    deg = int(deg)
    if deg < 2:
        avg_dist = -1.0
        avg_dist_std = 0.0
    else:
        avg_dist_per_run = []
        for _ in range(num_runs):
            uids_rand = np.random.choice(uids_valid, size=deg, replace=False)
            avg_dist_per_run.append(
                average_shortest_path_length(graph_user, uids_rand)
            )
        avg_dist = np.mean(avg_dist_per_run)
        avg_dist_std = np.std(avg_dist_per_run)
    avg_dist_per_deg[deg] = avg_dist
    avg_dist_std_per_deg[deg] = avg_dist_std
std_values = list(avg_dist_std_per_deg.values())
np.mean(std_values), np.max(std_values), np.min(std_values)


In [ ]:
for iid in range(num_items):
    deg = deg_i_valid[iid]
    print(
        "\t".join(
            [
                f"{iid}",
                f"{deg}",
                f"{avg_dist_per_item[iid]}",
                f"{avg_dist_per_deg[deg]}",
                f"{avg_dist_std_per_deg[deg]}",
            ]
        )
    )


In [ ]:
# adj_per_iid = {}
# for iid in range(num_items):
#     uids = user_history[:, iid].nonzero()
#     assert np.all(uids[1] == 0)
#     uids = uids[0]
#     rows, cols = [], []
#     for i in range(0, uids.size):
#         for j in range(i + 1, uids.size):
#             rows.append(uids[i])
#             rows.append(uids[j])
#             cols.append(uids[j])
#             cols.append(uids[i])
#     adj_per_iid.append(
#         sp.csr_matrix(
#             ([1.0] * len(rows), (rows, cols)),
#             shape=(num_users, num_users),
#             dtype=np.float32,
#         )
#     )
# len(adj_per_iid)


In [ ]:
# for iid in range(num_items):
#     assert adj_per_iid[iid].nnz == deg_i[iid] * (deg_i[iid] - 1)


In [ ]:
def gram_linear(X):
    return X @ X.T


def gram_rbf(X, threshold=1.0):
    gram = gram_linear(X)
    norms = torch.diag(gram)
    dist = -2 * gram + norms[:, None] + norms[None, :]
    dist_median = torch.median(dist).clamp_min_(torch.finfo(torch.float).tiny)
    rbf = torch.exp(-dist / (2 * threshold ** 2 * dist_median))
    return rbf


def center_gram(gram):
    means = torch.mean(gram, dim=0)
    means -= torch.mean(means) / 2
    gram -= means[:, None]
    gram -= means[None, :]

    return gram


def cka(X, Y, mode="linear", threshold=1.0):
    if mode == "linear":
        gram_X = gram_linear(X)
        gram_Y = gram_linear(Y)
    elif mode == "rbf":
        gram_X = gram_rbf(X, threshold)
        gram_Y = gram_rbf(Y, threshold)
    else:
        raise ValueError("Unknown mode {}".format(mode))

    gram_X = center_gram(gram_X)
    gram_Y = center_gram(gram_Y)
    scaled_hsic = gram_X.ravel() @ gram_Y.ravel()
    norm_X = torch.linalg.norm(gram_X)
    norm_Y = torch.linalg.norm(gram_Y)
    rst = scaled_hsic / (norm_X * norm_Y)

    return rst


def cca(X, Y):
    Qx, _ = torch.linalg.qr(X)
    Qy, _ = torch.linalg.qr(Y)
    rst = torch.linalg.norm(Qx.T @ Qy) ** 2 / min(X.shape[1], Y.shape[1])

    return rst


#### CCA


In [ ]:
# cca_per_iid = []
# for iid in range(num_items):
#     cca_per_iid.append(
#         cca(
#             torch.as_tensor(adj_mat.toarray(), dtype=torch.float),
#             torch.as_tensor(adj_per_iid[iid].toarray(), dtype=torch.float),
#         )
#     )
# cca_per_iid = np.asarray([v.item() for v in cca_per_iid], dtype=np.float32)
# cca_per_iid.shape, np.mean(cca_per_iid), np.std(cca_per_iid)


In [ ]:
# deg_i, cca_per_iid


#### CKA (Linear)


In [ ]:
# iid_cka_linear = {}
# for iid in range(num_items):
#     if deg_i[iid] == 1:
#         continue
#     iid_cka_linear[iid] = cka(
#         torch.as_tensor(adj_mat.toarray(), dtype=torch.float),
#         torch.as_tensor(adj_per_iid[iid].toarray(), dtype=torch.float),
#         mode="linear",
#     ).item()
# iid_cka_linear = sorted(
#     iid_cka_linear.items(), key=lambda x_y: x_y[1], reverse=True
# )
# iid_cka_linear[:10], iid_cka_linear[-10:]


In [ ]:
# for (iid, cka_coef) in iid_cka_linear:
#     print(f"{iid}\t{deg_i[iid]}\t{cka_coef}")


#### CKA (RBF)


In [ ]:
# threshold = 1.0
# iid_cka_rbf = {}
# for iid in range(num_items):
#     if deg_i[iid] == 1:
#         continue
#     iid_cka_rbf[iid] = cka(
#         torch.as_tensor(adj_mat.toarray(), dtype=torch.float),
#         torch.as_tensor(adj_per_iid[iid].toarray(), dtype=torch.float),
#         mode="rbf",
#         threshold=threshold,
#     ).item()
# iid_cka_rbf = sorted(iid_cka_rbf.items(), key=lambda x_y: x_y[1], reverse=True)
# iid_cka_rbf[:10], iid_cka_rbf[-10:]


In [ ]:
# threshold = 0.1
# iid_cka_rbf = {}
# for iid in range(num_items):
#     if deg_i[iid] == 1:
#         continue
#     iid_cka_rbf[iid] = cka(
#         torch.as_tensor(adj_mat.toarray(), dtype=torch.float),
#         torch.as_tensor(adj_per_iid[iid].toarray(), dtype=torch.float),
#         mode="rbf",
#         threshold=threshold,
#     ).item()
# iid_cka_rbf = sorted(iid_cka_rbf.items(), key=lambda x_y: x_y[1], reverse=True)
# iid_cka_rbf[:10], iid_cka_rbf[-10:]


In [ ]:
# threshold = 3.0
# iid_cka_rbf = {}
# for iid in range(num_items):
#     if deg_i[iid] == 1:
#         continue
#     iid_cka_rbf[iid] = cka(
#         torch.as_tensor(adj_mat.toarray(), dtype=torch.float),
#         torch.as_tensor(adj_per_iid[iid].toarray(), dtype=torch.float),
#         mode="rbf",
#         threshold=threshold,
#     ).item()
# iid_cka_rbf = sorted(iid_cka_rbf.items(), key=lambda x_y: x_y[1], reverse=True)
# iid_cka_rbf[:10], iid_cka_rbf[-10:]
